In [59]:
#used to store the decison tree

class TreeNode:
    def __init__(self, data,output):
        self.data = data
        self.children = {}
        self.output = output
        self.index = -1
        
    def add_child(self,feature_value,obj):
        self.children[feature_value] = obj

In [60]:
#our owm classifier (actual decision tree that'll print steps as well as store the obtained decision tree)

class D_T():
    def __init__(self):
        root=None
    def op(self,y):
        d={}
        s=set(y)
        for i in s:
            c=0
            for j in y:
                if i==j:
                    c+=1
                    d[i]=c
        return max(d)
    def classification(self,y):
        s=set(y)
        for i in s:
            c=0
            for j in y:
                if i==j:
                    c+=1
            print("Count of",i,':',c)
    def entropy(self,Y):
        classes = set(Y)
        value = 0
        for i in classes:
            p = (len(Y[Y==i])/len(Y))
            value = value - (p*np.log2(p))
        return value
    def findGainRatio(self,X,Y,selectedFeature):
        differentLabels = set(X[selectedFeature])
        entropyBeforeSplitting = self.entropy(Y)
        entropyAfterSplitting = 0
        splitInfo = 0
        for i in differentLabels:
            newNodeY = Y[(X[selectedFeature] == i)]
            weightOfSamples = (len(newNodeY)/len(Y))
            entropyAfterSplitting = entropyAfterSplitting + (weightOfSamples*self.entropy(newNodeY))
            splitInfo = splitInfo - (weightOfSamples*np.log2(weightOfSamples))
        gain = entropyBeforeSplitting - entropyAfterSplitting
        gainRatio = gain
        return gainRatio

    ###########################################################################################################################


    def dt(self,df,y,f,l):
        print()
        print("level :",l)

        self.classification(y)
        print("current entropy :",self.entropy(y))
        if len(f)==0 or (self.entropy(y)==0):
            print("reached leaf node")
            return TreeNode(None,self.op(y))
        sf=''
        max_value = -float('inf')
        for i in f:
            c=self.findGainRatio(df,y,i)
            if c>=max_value:
                sf = i
                max_value = c
        print("splitting on feature :",sf.upper(),"with gain ratio :",max_value)
        p=set(df[sf])
        f.remove(sf)
        current_node = TreeNode(sf,self.op(y))
        for i in p:
            a=(df[sf]==i)
            newdf=df[a]
            newy=y[a]
            node=self.dt(newdf,newy,f,l+1)
            current_node.add_child(i,node)
        return current_node
    def __predict_for(self,data,node):
            if len(node.children) == 0 :
                return node.output
            val = data[node.data]   
            if val not in node.children :
                return node.output
            return self.__predict_for(data,node.children[val])

    def predict(self,X):
            Y = np.array([0 for i in range(len(X))])
            for i in range(len(X)):
                Y[i] = self.__predict_for(X.iloc[i],self.root)
            return Y
    def score(self,X,Y):
            Y_pred = self.predict(X)
            count = 0
            for i in range(len(Y_pred)):
                if Y_pred[i] == Y[i]:
                    count+=1
            return count/len(Y_pred)
    def fit(self,x,y):
        f=list(df.columns)
        l=0
        self.root=self.dt(x,y,f,l)
        
        

# importing dataset and coverting it to labelled data

In [61]:
import numpy as np
import pandas as pd
from sklearn import datasets
iris = datasets.load_iris()
df=pd.DataFrame(iris.data)
y=(iris.target)
df.columns = ["sl", "sw", 'pl', 'pw']
def label(val, *boundaries):
    if (val < boundaries[0]):
        return 0
    elif (val < boundaries[1]):
        return 1
    elif (val < boundaries[2]):
        return 2
    else:
        return 3
def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df[old_feature_name].max()
    third = (maximum + second)/2
    return df[old_feature_name].apply(label, args= (first, second, third))
df['sl_labeled'] = toLabel(df, 'sl')
df['sw_labeled'] = toLabel(df, 'sw')
df['pl_labeled'] = toLabel(df, 'pl')
df['pw_labeled'] = toLabel(df, 'pw')
df.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)
f=list(df.columns)

# using our algo on the labelled dataset

In [62]:
clf=D_T()

In [63]:
clf.fit(df,y)


level : 0
Count of 0 : 50
Count of 1 : 50
Count of 2 : 50
current entropy : 1.584962500721156
splitting on feature : PW_LABELED with gain ratio : 1.2627308217896138

level : 1
Count of 0 : 50
current entropy : 0.0
reached leaf node

level : 1
Count of 1 : 10
current entropy : 0.0
reached leaf node

level : 1
Count of 1 : 40
Count of 2 : 16
current entropy : 0.863120568566631
splitting on feature : PL_LABELED with gain ratio : 0.3107100419019655

level : 2
Count of 1 : 1
current entropy : 0.0
reached leaf node

level : 2
Count of 1 : 39
Count of 2 : 8
current entropy : 0.6581912658132185
splitting on feature : SL_LABELED with gain ratio : 0.15790810784426046

level : 3
Count of 2 : 1
current entropy : 0.0
reached leaf node

level : 3
Count of 1 : 14
current entropy : 0.0
reached leaf node

level : 3
Count of 1 : 23
Count of 2 : 7
current entropy : 0.783776947484701
splitting on feature : SW_LABELED with gain ratio : 0.08807926473635475

level : 4
Count of 1 : 3
Count of 2 : 1
current e

In [64]:
y_pred=clf.predict(df)

In [65]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1,
       1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [66]:
clf.score(df,y)

0.8866666666666667